In [1]:
# from google.colab import drive
# import os

# drive.mount('/content/drive')


In [2]:
# os.chdir('drive/My Drive/')  # Change this path to your desired directory


In [3]:
# ! pip install kaggle


In [4]:
# # Upload your kaggle.json API key file to your Colab environment.
# from google.colab import files
# uploaded = files.upload()

# # Move the kaggle.json file to the appropriate directory and set the correct permissions.
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [5]:
# !kaggle datasets download -d ishanikathuria/handwritten-signature-datasets


In [6]:
# import zipfile

# # Replace 'dataset.zip' with the actual name of your zip file
# with zipfile.ZipFile('handwritten-signature-datasets.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/dataset')

In [7]:
import tensorflow as tf


a=tf.config.experimental.list_physical_devices("GPU")
if a :
  for b in a:
    tf.config.experimental.set_memory_growth(b,True)
  device=a[0]

In [8]:
# import os
# import random
# from itertools import combinations
# import numpy as np

# # # Mount Google Drive (if your dataset is stored there)
# # from google.colab import drive

# # drive.flush_and_unmount()

# # drive.mount('/dataset/CEDAR/')

# # os.chdir('../../')  # Change this path to your desired directory


# # Define the path to your dataset folder on Google Drive
# dataset_folder = '/content/dataset/CEDAR/CEDAR'  # Adjust the path to your dataset

# # Check if the dataset folder exists
# if not os.path.exists(dataset_folder):
#     print(f"The dataset folder '{dataset_folder}' does not exist.")
# else:
#     print(f"Dataset folder '{dataset_folder}' found.")


In [9]:
import os
import random
from itertools import combinations
import numpy as np

# Define the path to your dataset folder
dataset_folder = 'archive\CEDAR\CEDAR'

# Define the percentage of data for the training set
train_percentage = 0.7

# Initialize lists to store image pairs and their labels
train_image_pairs = []
train_labels = []
test_image_pairs = []
test_labels = []

# Iterate through each folder in the dataset
for folder_name in os.listdir(dataset_folder):
    folder_path = os.path.join(dataset_folder, folder_name)

    # Create a list of original and forged images in the folder
    original_images = []
    forged_images = []
    for file_name in os.listdir(folder_path):
        if file_name.startswith('original'):
            original_images.append(file_name)
        elif file_name.startswith('forgeries'):
            forged_images.append(file_name)

    # Shuffle the original and forged images to ensure randomness
    random.shuffle(original_images)
    random.shuffle(forged_images)

    # # Determine how many images to use for training and testing based on the percentage
    # num_train_original = int(train_percentage * len(original_images))
    # num_test_original = len(original_images) - num_train_original
    # num_train_forged = int(train_percentage * len(forged_images))
    # num_test_forged = len(forged_images) - num_train_forged

    # Generate pairs for original-original and original-forged combinations
    original_original_pairs = list(combinations(original_images, 2))
    original_forged_pairs = [(original_image, forged_image) for original_image in original_images for forged_image in forged_images]

    # Shuffle the pairs for randomness
    random.shuffle(original_original_pairs)
    random.shuffle(original_forged_pairs)

    # Select an equal number of original-original and original-forged pairs for training
    original_original_pairs = original_original_pairs[:50]
    original_forged_pairs = original_forged_pairs[:len(original_original_pairs)]


    train_number=int(train_percentage*len(original_original_pairs))

    train_original_original_pairs=original_original_pairs[:train_number]
    train_original_forged_pairs=original_forged_pairs[:train_number]
    # Assign labels (0 for original-original and 1 for original-forged)
    train_labels.extend([0] * len(train_original_original_pairs) + [1] * len(train_original_forged_pairs))


    # Extend the training sets
    train_image_pairs.extend([(os.path.join(folder_path, pair[0]), os.path.join(folder_path, pair[1])) for pair in train_original_original_pairs + train_original_forged_pairs])

    # Select the remaining pairs for testing
    test_original_original_pairs = original_original_pairs[train_number:]
    test_original_forged_pairs = original_forged_pairs[train_number:]

    # Assign labels for testing
    test_labels.extend([0] * len(test_original_original_pairs) + [1] * len(test_original_forged_pairs))

    # Extend the testing sets
    test_image_pairs.extend([(os.path.join(folder_path, pair[0]), os.path.join(folder_path, pair[1])) for pair in test_original_original_pairs + test_original_forged_pairs])

# Now you have the training and testing sets with image pairs and labels as per your requirements


In [10]:
# Print a few examples from test_image_pairs and test_labels
print("Sample test_image_pairs:")
for i in range(min(5, len(test_image_pairs))):  # Print up to 5 examples
    print(test_image_pairs[i], "Label:", test_labels[i])

# Print the total number of test_image_pairs and test_labels
print("Number of test_image_pairs:", len(test_image_pairs))
print("Number of test_labels:", len(test_labels))

# Print a few examples from train_image_pairs and train_labels
print("\nSample train_image_pairs:")
for i in range(min(5, len(train_image_pairs))):  # Print up to 5 examples
    print(train_image_pairs[i], "Label:", train_labels[i])

# Print the total number of train_image_pairs and train_labels
print("Number of train_image_pairs:", len(train_image_pairs))
print("Number of train_labels:", len(train_labels))


Sample test_image_pairs:
('archive\\CEDAR\\CEDAR\\1\\original_1_9.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_16.png') Label: 0
('archive\\CEDAR\\CEDAR\\1\\original_1_4.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_13.png') Label: 0
('archive\\CEDAR\\CEDAR\\1\\original_1_20.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_7.png') Label: 0
('archive\\CEDAR\\CEDAR\\1\\original_1_21.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_17.png') Label: 0
('archive\\CEDAR\\CEDAR\\1\\original_1_24.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_14.png') Label: 0
Number of test_image_pairs: 1650
Number of test_labels: 1650

Sample train_image_pairs:
('archive\\CEDAR\\CEDAR\\1\\original_1_13.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_12.png') Label: 0
('archive\\CEDAR\\CEDAR\\1\\original_1_24.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_8.png') Label: 0
('archive\\CEDAR\\CEDAR\\1\\original_1_23.png', 'archive\\CEDAR\\CEDAR\\1\\original_1_3.png') Label: 0
('archive\\CEDAR\\CEDAR\\1\\original_1_8.png', 'archive\\CED

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.applications import VGG19
from keras.models import Model
from keras.callbacks import EarlyStopping

# Define a function to preprocess the image pairs
def preprocess_image(image_path):
    # Load, resize, and normalize the image
    image = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = keras.preprocessing.image.img_to_array(image) / 255.0
    return image

# Load the pre-trained VGG-19 model without the top classification layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the first 5 layers
for layer in base_model.layers[:5]:
    layer.trainable = False

# Define the Siamese CNN model
def create_siamese_model(input_shape):
    # Create the left and right inputs
    input_left = keras.layers.Input(shape=input_shape)
    input_right = keras.layers.Input(shape=input_shape)

    # Generate the embeddings for the left and right inputs using the base VGG-19 model
    embedding_left = base_model(input_left)
    embedding_right = base_model(input_right)

    # Flatten the embeddings
    embedding_left = keras.layers.Flatten()(embedding_left)
    embedding_right = keras.layers.Flatten()(embedding_right)

    # Calculate the L1 distance between the embeddings
    L1_distance = keras.layers.Lambda(lambda embeddings: tf.abs(embeddings[0] - embeddings[1]))([embedding_left, embedding_right])

    # Output layer
    output_layer = keras.layers.Dense(1, activation='sigmoid')(L1_distance)

    # Create the Siamese model
    siamese_model = keras.models.Model(inputs=[input_left, input_right], outputs=output_layer)

    return siamese_model

# Create and compile the Siamese model
input_shape = (224, 224, 3)
siamese_model = create_siamese_model(input_shape)
siamese_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# The rest of your code for loading data and training the model remains the same.

import numpy as np

# Convert your image pairs to NumPy arrays
train_image_pairs_left = np.array([preprocess_image(pair[0]) for pair in train_image_pairs])
train_image_pairs_right = np.array([preprocess_image(pair[1]) for pair in train_image_pairs])

# Convert your labels to a NumPy array
train_labels = np.array(train_labels)

# Similarly, preprocess and convert test image pairs and labels
test_image_pairs_left = np.array([preprocess_image(pair[0]) for pair in test_image_pairs])
test_image_pairs_right = np.array([preprocess_image(pair[1]) for pair in test_image_pairs])
test_labels = np.array(test_labels)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, restore_best_weights=True)

# Train the Siamese model
siamese_model.fit(
    [train_image_pairs_left, train_image_pairs_right],
    train_labels,
    batch_size=8,
    epochs=10,
    validation_data=([test_image_pairs_left, test_image_pairs_right], test_labels),
    callbacks=[early_stopping]
)

# Save the Siamese model
siamese_model.save('vgg_19')
